# Aave V3 USDC Borrow Incentives Analysis on Base

This notebook analyzes USDC borrow incentives on Aave V3 deployed on Base network (Chain ID: 8453). We'll fetch the current borrow APY and any available incentives to calculate the effective borrowing cost.


### Step 1: Convert Token Address to Lowercase

USDC token address needs to be in lowercase format for API consistency.


In [ ]:
# Convert USDC token address to lowercase for API consistency
"0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913".lower()

'0x833589fcd6edb6e08f4c7c32d4f71b54bda02913'

### Step 2: Setup API Configuration and GraphQL Query

Configure the Aave V3 API endpoint and create a GraphQL query to fetch:
- Underlying token information (USDC)
- Current borrow APY
- Available incentives (Merit and Aave rewards)

In [41]:
import requests
import json
from decimal import Decimal

univ3_url =  "https://gateway-arbitrum.network.thegraph.com/api/2f2a4cb66c9247a50de53fc9a2d95f29/subgraphs/id/5zvR82QoaXYFyDEKLZ9t6v9adgnptxYpKpSbxtgVENFV"
url = "https://api.v3.aave.com/graphql"

# params = {
#     'query': '{pool(id:"' + pool_address + '", block: {number: ' + str(block) + '}){tick,sqrtPrice,liquidity,ticks(first:1000,orderBy:tickIdx,where:{liquidityNet_not:"0"}){tickIdx,liquidityGross,liquidityNet}}}'
# }

# GraphQL query
query = """
query BorrowIncentivesBaseUSDC($request: ReserveRequest!) {
  reserve(request: $request) {
    underlyingToken { symbol address }
    borrowInfo {
      apy {
        decimals
        formatted
        raw
        value
      }
    }
    incentives {
      __typename
      ... on MeritBorrowIncentive {
        claimLink
        borrowAprDiscount {
          value       # decimal fraction, e.g. 0.0123 => 1.23% APR
          formatted   # human string, e.g. "1.23%"
          raw
          decimals
        }
      }
      ... on AaveBorrowIncentive {
        rewardTokenSymbol
        borrowAprDiscount {
          value
          formatted
          raw
          decimals
        }
      }
    }
  }
}
"""

variables = {
  "request": {
    "chainId": 8453,
    "market": "0xa238dd80c259a72e81d7e4664a9801593f98d1c5",
    "underlyingToken": "0x833589fcd6edb6e08f4c7c32d4f71b54bda02913"
  }
}



## Step 3: Execute API Request

Send the GraphQL query to Aave V3 API to fetch token reserve data on a network.

In [ ]:
response = requests.post(url, 
                        data=json.dumps({"query": query, "variables": variables}), 
                        headers={"Content-Type": "application/json"})

## Step 4: Check Response Status

Verify that the API request was successful (with status code 200).

In [43]:
response.status_code

200

## Step 5: Parse API Response

Extract the JSON response data containing:
- Token symbol and address
- Current borrow APY
- Available incentives (Aave and Merit borrow discount)

In [45]:
data = response.json()
data

{'data': {'reserve': {'underlyingToken': {'symbol': 'USDC',
    'address': '0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913'},
   'borrowInfo': {'apy': {'decimals': 27,
     'formatted': '7.82',
     'raw': '78192067667154668466057440',
     'value': '0.07819206766715466846605744'}},
   'incentives': [{'__typename': 'MeritBorrowIncentive',
     'claimLink': 'https://apps.aavechan.com/merit/base-borrow-usdc',
     'borrowAprDiscount': {'value': '0.02',
      'formatted': '2.00',
      'raw': '20000000000000000000000000',
      'decimals': 27}}]}}}

## Step 6: Calculate Total Incentive APR

Sum up all available incentives to get the total APR discount.

In [48]:
total_incentive_apr = sum([Decimal(i['borrowAprDiscount']['value']) for i in data['data']['reserve']['incentives']])
protocol_apy = float(data['data']['reserve']['borrowInfo']['apy']['value'])
total_incentive_apr

Decimal('0.02')

## Step 7: Convert APR to APY

Convert the incentive APR to APY using compound interest formula: `(1 + APR/365)^365 - 1`

In [49]:
incentives_apr = (Decimal(1) + total_incentive_apr/Decimal(365))**Decimal(365) - Decimal(1)
incentives_apr

Decimal('0.020200781032895841595951979')

## Step 8: Calculate Effective Borrowing Cost

Calculate the net borrowing cost after applying incentives:
- **Protocol APY**: 7.82% (base borrowing rate)
- **Incentive APY**: 2.02% (discount from Merit rewards)
- **Effective Rate**: 5.80% (net cost after incentives)

In [56]:
incentives_apy_in_pct = float(f"{(incentives_apr*100):.2f}")
protocol_apy_in_pct = float(f"{(protocol_apy*100):.2f}")
print(f"Protocol APY: {protocol_apy_in_pct}%")
print(f"Borrow incentive APY of USDC on Base: {incentives_apy_in_pct}%")

print(f"Total APY (Borrow Rate): {(protocol_apy_in_pct - incentives_apy_in_pct):.2f}%")


Protocol APY: 7.82%
Borrow incentive APY of USDC on Base: 2.02%
Total APY (Borrow Rate): 5.80%
